In [1]:
import numpy as np
import pandas as pd

from skmultilearn.dataset import load_dataset
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import BRkNNaClassifier
from skmultilearn.adapt import MLkNN
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelD

from sklearn.svm import SVC

import sklearn.metrics as metrics
import scipy.sparse as sparse

import copy

## Import Dataset

In [2]:
X_train, y_train, feature_names, label_names = load_dataset('emotions', 'train')
X_test, y_test, _, _ = load_dataset('emotions', 'test')

emotions:train - exists, not redownloading
emotions:test - exists, not redownloading


## Determine a Model

Rakel: http://scikit.ml/api/skmultilearn.ensemble.rakeld.html#skmultilearn.ensemble.RakelD

In [3]:
#clf = BinaryRelevance(classifier=SVC(), require_dense=None)
#clf = ClassifierChain(classifier=SVC(), require_dense=None)
#clf = LabelPowerset(classifier=SVC(), require_dense=None)
clf = BRkNNaClassifier(k=3)
#clf = MLkNN(k=5)
clf = RakelD(
#    base_classifier=GaussianNB(),
#    base_classifier_require_dense=[True, True],
#    labelset_size=4
#)

SyntaxError: unexpected EOF while parsing (<ipython-input-3-e5ed1ee73d4b>, line 10)

## Transform Results

In [ ]:
def trans_result(y_testing, pred):
    col_count = y_testing.shape[1]
    y_test_df = pd.DataFrame(y_testing.todense(),columns=["L" + str(x+1) for x in range(col_count)])
    pred_df = pd.DataFrame(pred.todense(),columns=["L" + str(x+1) for x in range(col_count)])
    z=[]
    for col in y_test_df.columns:
        z.append({"name": col,
              "precision": metrics.precision_score(y_test_df[col], pred_df[col]),
              "recall": metrics.precision_score(y_test_df[col], pred_df[col]),
              "f1": metrics.f1_score(y_test_df[col], pred_df[col])})
    return pd.DataFrame(z)

In [ ]:
prediction = clf.fit(X_train, y_train).predict(X_test)
result_df = trans_result(y_test,prediction)
result_df

## Exp1 : Remove an label and retrain

In [ ]:
def remove_a_label(dataset, index):
    dataset = dataset.todense()
    j=[]
    for ele in dataset:
        j.append(np.delete(ele,2))
    return sparse.lil_matrix(np.array(j))

In [ ]:
y_train_1 = remove_a_label(copy.deepcopy(y_train),2)
y_test_1 = remove_a_label(copy.deepcopy(y_test),2)

In [ ]:
prediction_1 = clf.fit(X_train, y_train_1).predict(X_test)
result_df_1 = trans_result(y_test_1,prediction_1)
result_df_1

## Exp2: Add an label and retrain

In [ ]:
def add_a_label(dataset):
    dataset = dataset.todense()
    j=[]
    for ele in dataset:
        new_label = 1 if np.random.rand() > 0.5 else 0
        j.append(np.append(np.array(ele),new_label))
    return sparse.lil_matrix(np.array(j))

In [ ]:
y_train_2 = add_a_label(copy.deepcopy(y_train))
y_test_2 = add_a_label(copy.deepcopy(y_test))

In [ ]:
prediction_2 = clf.fit(X_train, y_train_2).predict(X_test)
result_df_2 = trans_result(y_test_2,prediction_2)
result_df_2